Model is trained with 5 actions: CricketShot,ShavingBeard,PlayingCello,TennisSwing,Punch

In [1]:
!pip install -q git+https://github.com/tensorflow/docs


In [2]:
!wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
!tar xf ucf101_top5.tar.gz


In [1]:
#import libraries
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os


2022-11-05 17:47:46.304894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 17:47:46.784439: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-05 17:47:47.665541: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-05 17:47:47.665736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
#define hyper parameters
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048


In [3]:
#data prep
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)


Total videos for training: 594
Total videos for testing: 224


video_name           tag
489   v_TennisSwing_g09_c07.avi   TennisSwing
123  v_PlayingCello_g08_c06.avi  PlayingCello
521   v_TennisSwing_g14_c04.avi   TennisSwing
59    v_CricketShot_g16_c04.avi   CricketShot
284         v_Punch_g14_c06.avi         Punch
254         v_Punch_g10_c03.avi         Punch
541   v_TennisSwing_g17_c03.avi   TennisSwing
160  v_PlayingCello_g14_c02.avi  PlayingCello
271         v_Punch_g12_c07.avi         Punch
139  v_PlayingCello_g11_c02.avi  PlayingCello

In [4]:
#helper function
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


2022-11-05 17:47:55.724526: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-11-05 17:47:55.724622: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: anagha-OMEN-Laptop-15-ek0xxx
2022-11-05 17:47:55.724645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: anagha-OMEN-Laptop-15-ek0xxx
2022-11-05 17:47:55.724897: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.85.2
2022-11-05 17:47:55.724959: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2022-11-05 17:47:55.724979: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 515.65.1 does not match DSO version 510.85.2 -- cannot find working devices in this configuration
2022-11-05 17:47:55.725607: I tensorflow/core/platform/c

In [5]:
#printing labels
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())


['CricketShot', 'PlayingCello', 'Punch', 'ShavingBeard', 'TennisSwing']


In [6]:
#data processing utility
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 51ms/step


...

1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 34ms/step
Frame features in train set: (594, 20, 2048)
Frame masks in train set: (594, 20)


In [8]:
#sequence model
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()


Epoch 1/10
12/13 [==========================>...] - ETA: 0s - loss: 1.4548 - accuracy: 0.3906
Epoch 1: val_loss improved from inf to 1.38741, saving model to /tmp/video_classifier
13/13 [==============================] - 7s 161ms/step - loss: 1.4453 - accuracy: 0.4048 - val_loss: 1.3874 - val_accuracy: 0.2346
Epoch 2/10
13/13 [==============================] - ETA: 0s - loss: 1.1451 - accuracy: 0.6217
Epoch 2: val_loss improved from 1.38741 to 1.19354, saving model to /tmp/video_classifier
13/13 [==============================] - 0s 20ms/step - loss: 1.1451 - accuracy: 0.6217 - val_loss: 1.1935 - val_accuracy: 0.3464
Epoch 3/10
13/13 [==============================] - ETA: 0s - loss: 1.0205 - accuracy: 0.6771
Epoch 3: val_loss improved from 1.19354 to 1.18526, saving model to /tmp/video_classifier
13/13 [==============================] - 0s 19ms/step - loss: 1.0205 - accuracy: 0.6771 - val_loss: 1.1853 - val_accuracy: 0.3464
Epoch 4/10
13/13 [==============================] - ETA: 0s -

In [7]:
prob={}
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    global prob
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
        prob[class_vocab[i]]=probabilities[i] * 100
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
'''def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])'''

'def to_gif(images):\n    converted_images = images.astype(np.uint8)\n    imageio.mimsave("animation.gif", converted_images, fps=10)\n    return embed.embed_file("animation.gif")\n\n\ntest_video = np.random.choice(test_df["video_name"].values.tolist())\nprint(f"Test video path: {test_video}")\ntest_frames = sequence_prediction(test_video)\nto_gif(test_frames[:MAX_SEQ_LENGTH])'

In [9]:
#flask code
from flask import Flask
import os
from flask import render_template,url_for,render_template,request,redirect
app=Flask(__name__)
from werkzeug.utils import secure_filename
import webbrowser
from threading import Timer
ct=0
def open_browser():
    webbrowser.open_new('http://127.0.0.1:2000/')

@app.route("/")
def initial():
    return redirect(url_for("home"))
#HOME PAGE
@app.route("/home",methods=["POST","GET"])
def home():
    global prob
    ct=0
    if request.method=="POST":
        button_clicked=request.form["action"]
        if button_clicked =="SUBMIT":
            f=request.files['myfile']
            f.save(secure_filename(f.filename))#this command helps to save the file in templates folder in the current directory.To give path use the instruction in next line
            test_video = f.filename
            test_frames = sequence_prediction(test_video)
            return redirect(url_for("output"))
    else:
        return render_template("home.html")

 #OUTPUT PAGE   
@app.route("/output",methods=["POST","GET"])
def output():
    global prob
    v=max(prob, key=prob.get)
    return render_template("output.html",count=v)#specify file names here
if __name__=="__main__":
    Timer(1,open_browser).start();
    app.run(port=2000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:2000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Nov/2022 18:22:36] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [05/Nov/2022 18:22:36] "GET /home HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [05/Nov/2022 18:22:43] "POST /home HTTP/1.1" 302 -
127.0.0.1 - - [05/Nov/2022 18:22:43] "GET /output HTTP/1.1" 200 -


  CricketShot: 20.91%
  PlayingCello: 20.53%
  Punch: 20.44%
  ShavingBeard: 19.34%
  TennisSwing: 18.78%
